# Задание 2. Фестский диск

## 1) Импорт библиотек

In [1]:
import numpy as np
from itertools import chain

## 2) Входные данные

Исходный текст Фестского диска представлен ниже в формате

- $char := n \in \mathbb{N}$
- $word := (char),word$
- $sentence := (word),sentence$
- $text := (sentence),text$

Каждое слово - список символов, предложение - список слов, а текст - список предложений.

In [2]:
a_side = [
    [[2,12,13,1,18],],
    [[24,40,12], [29,45,7],],
    [[29,29,34], [2,12,4,40,33], [27,45,7,12], [27,44,8], [2,12,6,18,255], [31,26,35], [2,12,41,19,35], [1,41,40,7] ,[2,12,32,23,38],],
    [[39,11], [2,27,25,10,23,18], [28,1],],
    [[2,12,31,26],],
    [[2,12,27,27,35,37,21] ,[33,23] ,[2,12,31,26],],
    [[2,27,25,10,23,18], [28,1],],
    [[2,12,31,26],],
    [[2,12,27,14,32,18,27], [6,18,17,19], [31,26,12], [2,12,13,1], [23,19,35],],
    [[10,3,38], [2,12,27,27,35,37,21], [13,1], [10,3,38],],
]

b_side = [
    [[2,12,22,40,7], [27,45,7,35], [2,37,23,5],],
    [[22,25,27], [33,24,20,12], [16,23,18,43],],
    [[13,1,39,33], [15,7,13,1,18], [22,37,42,25], [7,24,40,35], [2,26,36,40], [27,25,38,1], [29,24,24,20,35], [16,14,18], [29,33,1], [6,35,32,39,33], [2,9,27,1], [29,36,7,8],],
    [[29,8,13], [29,45,7],],
    [[22,29,36,7,8],],
    [[27,34,23,25], [7,18,35], [7,45,7],],
    [[7,23,18,24], [22,29,36,7,8],],
    [[9,30,39,18,7], [2,6,35,23,7], [29,34,23,25], [45,7],],
]

## 3) Расчет средних значений длины слова и предложения в языке:

In [3]:
a_sent_len_mean = np.mean([len(sentence) for sentence in a_side])
b_sent_len_mean = np.mean([len(sentence) for sentence in b_side])
sent_len_mean = int(np.round(.5 * (a_sent_len_mean + b_sent_len_mean)))

a_word_len_mean = np.mean([np.mean([len(word) for word in sentence]) for sentence in a_side])
b_word_len_mean = np.mean([np.mean([len(word) for word in sentence]) for sentence in b_side])
word_len_mean = int(np.round(.5 * (a_word_len_mean + b_word_len_mean)))

In [4]:
print(sent_len_mean)
print(word_len_mean)

3
4


## 4) N-граммная энтропия

Введем функцию

$\Large\sum\limits_{i} p(i) log_2p(i)$

для вычисления $N$-граммной энтропии:

In [5]:
def n_entropy(characters, n):
    ngrams = [tuple(characters[i:i+n]) for i in range(len(characters)-n)]
    
    counts = dict()
    for ng in ngrams:
        if not (ng in counts.keys()):
            counts[ng] = 0
        counts[ng] += 1
    
    ng_amounts = np.array(list(counts.values()))
    total_ng = int(np.sum(ng_amounts))
    ng_distr = ng_amounts/total_ng
    
    return -np.dot(np.log2(ng_distr), ng_distr)

Далее текст с обеих сторон диска конкатенируется в непрерывную цепочку символов.
После этого выполняется поиск предельного значения энтропии:
$\lim_{N\rightarrow \infty} F_N$

In [6]:
flat_a = list(chain.from_iterable(a_side))
flat_a = list(chain.from_iterable(flat_a))

flat_b = list(chain.from_iterable(b_side))
flat_b = list(chain.from_iterable(flat_b))

flat_text = flat_a + flat_b

for i in range(1, 25):
    print('Entropy (N = {:2d}): {:.4f}'.format(i, n_entropy(flat_text, i) - n_entropy(flat_text, i - 1)))

Entropy (N =  1): 5.0111
Entropy (N =  2): 2.0699
Entropy (N =  3): 0.5080
Entropy (N =  4): 0.0946
Entropy (N =  5): 0.0585
Entropy (N =  6): 0.0388
Entropy (N =  7): 0.0105
Entropy (N =  8): 0.0106
Entropy (N =  9): 0.0021
Entropy (N = 10): 0.0022
Entropy (N = 11): 0.0022
Entropy (N = 12): 0.0022
Entropy (N = 13): 0.0023
Entropy (N = 14): 0.0023
Entropy (N = 15): 0.0024
Entropy (N = 16): 0.0024
Entropy (N = 17): -0.0064
Entropy (N = 18): -0.0064
Entropy (N = 19): -0.0065
Entropy (N = 20): -0.0065
Entropy (N = 21): -0.0065
Entropy (N = 22): -0.0065
Entropy (N = 23): -0.0066
Entropy (N = 24): -0.0066


Энтропия быстро убывает и становится незначительной при $N \in [4,\infty)$

## 5) Вычисление расстояния единственности шифра

In [7]:
# H(k) - энтропия ключа, считается от средней длины слова
key_ent = n_entropy(flat_text, word_len_mean) - n_entropy(flat_text, word_len_mean - 1)

# R(M) - абсолютная энтропия
absolute_ent = np.log2(len(set(flat_text)))
# r(s) - фактическая энтропия, считается от средней длины предложения
real_ent = n_entropy(flat_text, sent_len_mean) - n_entropy(flat_text, sent_len_mean - 1)

# D = (R(M) - r(s)) - избыточность
redundancy = absolute_ent - real_ent

# U = H(k)/D - расстояние единственности
uniq_dist = key_ent/redundancy

print('H = {:.4f}'.format(key_ent))
print('R = {:.4f}'.format(absolute_ent))
print('r = {:.4f}'.format(real_ent))
print('D = (R - r) = {:.4f}'.format(redundancy))
print('U = H/D = {:.4f}'.format(uniq_dist))

H = 0.0946
R = 5.5236
r = 0.5080
D = (R - r) = 5.0155
U = H/D = 0.0189


## 6) Вывод

Расстояние единственности в результате расчетов получилось практически нулевым, т.е. для восстановления ключа нужен максимум один символ зашифрованного сообщения.

Однако неизвестен фактический размер алфавита и синтаксис языка - вполне возможно, что косые черты не являются знаками пунктуации, или что предложения, записанные на дисках, короче, чем среднее по размеру предложение в данном языке (ввиду того, что на диске мало места).

Имея лишь один экземпляр с текстом на данном языке, невозможно делать предположения о сложности дешифрования.